In [1]:
from enum import Enum
import random
import skills

In [2]:
class Pool:
    def __init__(self, maximum=0, edge=0):
        self.max = maximum
        self.current = maximum
        self.edge = edge
    def __repr__(self):
        return "{}/{}".format(self.current, self.max)
    def reduce(self, amount):
        if(self.current > 0):
            difference = self.current - amount
            if(difference >= 0):
                self.current -= amount
                return 0
            else:
                self.current = 0
                return abs(difference)
            #self.current -= amount
            #self.current = max(self.current, 0)
            #return self.current
        else:
            return amount
    def increase(self, amount):
        if(self.current < self.max):
            self.current += amount
            self.current = min(self.current, self.max)
    def effort(self, num):
        spent = max(0, (num*3) - self.edge)
        if(spent <= self.current):
            self.reduce(spent)
            return True
        else:
            print("Not enough to spend {} effort.\nCurrent: {}\nRequired: {}".format(num, self.current, spent))
            return False

In [3]:
class Stat(Enum):
    Might = 0
    Speed = 1
    Intellect = 2

In [4]:
class Weapon(Enum):
    Light = 2
    Medium = 4
    Heavy = 6

# Skills List

* Performance
* Bluff
* Insight
* Diplomacy
* Athletics: Might
* Athletics: Speed
* Perception
* Healing
* Survival
* Stealth
* Investigation



* Might Attack
* Speed Attack (Ranged)
* Speed Attack (Melee)
* Intellect Attack
* Might Defense
* Speed Defense
* Intellect Defense
* Initiative


* Numenera/Magic/Occult
* Craft: _

**Social**

* Perform: Stat.Intellect

* Bluff: Stat.Intellect

* Insight: Stat.Intellect

* Diplomacy: Stat.Intellect



**Adventuring**

* Athletics: Stat.Might

* Acrobatics: Stat.Speed

* Perception: Stat.Intellect

* Healing: Stat.Intellect

* Survival: Stat.Intellect

* Stealth: Stat.Speed

* Investigation: Stat.Intellect

* Thievery: Stat.Speed



**Numenera**

* Mechanical: Stat.Intellect

* Biological: Stat.Intellect

* Crystalline: Stat.Intellect

* Energy: Stat.Intellect

* Psychic: Stat.Intellect

* Psychic: Stat.Intellect



**Crafting**

* Weapons: Stat.Intellect

* Armor: Stat.Intellect

* Devices: Stat.Intellect

* Tools: Stat.Intellect

* Consumables: Stat.Intellect

* Breeding: Stat.Intellect

* Animal Raising: Stat.Intellect



**Combat**

* Attack: Stat.Might

* Attack: Stat.Speed

* Attack: Stat.Intellect

* Defense: Stat.Might

* Defense: Stat.Speed

* Defense: Stat.Intellect

* Initiative: Stat.Speed

* Light Weapons: Stat.Speed

* Medium Weapons: Stat.Might

* Heavy Weapons: Stat.Might

In [5]:
class Character:
    def __init__(self, name='', 
                 might=0, 
                 speed=0, 
                 intellect=0, 
                 mightEdge=0, 
                 speedEdge=0, 
                 intellectEdge=0,
                 inputSkills = [],
                 weapon = Weapon.Light):
        self.name = name
        self.might = Pool(might, mightEdge)
        self.speed = Pool(speed, speedEdge)
        self.intellect = Pool(intellect, intellectEdge)
        self.adventureSkills = []
        self.combatSkills = []
        self.craftingSkills = []
        self.numeneraSkills = []
        self.socialSkills = []
        for skill in inputSkills:
            if(skills.isSkill(skill)):
                if(skills.isAdventuringSkill(skill)):
                    self.adventureSkills.append(skill)
                elif(skills.isCombatSkill(skill)):
                    self.combatSkills.append(skill)
                elif(skills.isCraftingSkill(skill)):
                    self.craftingSkills.append(skill)
                elif(skills.isNumeneraSkill(skill)):
                    self.numeneraSkills.append(skill)
                elif(skills.isSocialSkill(skill)):
                    self.socialSkills.append(skill)
        self.skills = self.adventureSkills + \
                      self.combatSkills + \
                      self.craftingSkills + \
                      self.numeneraSkills + \
                      self.socialSkills
        self.weapon = weapon
        #self.damage = 2
    def roll(self):
        return random.randint(1,20)
    def Challenge(self, level=0, skills=[], stat=Stat.Might, effort=0, assets=0):
        skillCounter = 0
        for skill in skills:
            if skill in self.skills:
                skillCounter += 1
        if(self.Effort(stat, effort)):
            target = (level - skillCounter - effort - assets)*3
        else:
            target = (level - skillCounter - assets)*3
        print('Target number: {}'.format(target))
        if(target <= 0):
            return True
        else:
            roll = self.roll()
            print('{} rolled {}.'.format(self.name, roll))
            return roll >= target
    def Effort(self, stat=Stat.Might, num=1):
        if(stat == Stat.Might):
            return self.might.effort(num)
        elif(stat == Stat.Speed):
            return self.speed.effort(num)
        elif(stat == Stat.Intellect):
            return self.intellect.effort(num)
    def isDead(self):
        return self.might.current==0 and \
               self.speed.current == 0 and \
               self.intellect.current == 0
    def Attack(self, creature, stat=Stat.Speed, effort=0, assets=0):
        print('{} attacks {}.'.format(self.name, creature.name))
        result = self.Challenge(level=creature.level, skills=['{} Attack'.format(stat.name), '{} Weapons'.format(self.weapon.name)], effort=effort, assets=assets)
        if(result):
            creature.hp -= self.weapon.value
    def __repr__(self):
        adventuring = 'ADVENTURING SKILLS:\n' + '\n'.join(self.adventureSkills) + '\n'
        combat = '\nCOMBAT SKILLS:\n' + '\n'.join(self.combatSkills) + '\n'
        crafting = '\nCRAFTING SKILLS:\n' + '\n'.join(self.craftingSkills) + '\n'
        numenera = '\nNUMENERA SKILLS:\n' + '\n'.join(self.numeneraSkills) + '\n'
        social = '\nSOCIAL SKILLS:\n' + '\n'.join(self.socialSkills) + '\n'
        skills = '\n' + adventuring + combat + crafting + numenera + social
        return "Name: {}\nMight: {}\nSpeed: {}\nIntellect: {}\nSkills: {}".format(self.name, self.might, self.speed, self.intellect, skills)

In [6]:
Stat.Speed.name

'Speed'

In [7]:
theron = Character('Theron', 18, 12, 8, 1, 1, 0, ['Might Defense', 'Might Attack', 'Speed Attack', 'Athletics', 'Heavy Weapons'])
theron

Name: Theron
Might: 18/18
Speed: 12/12
Intellect: 8/8
Skills: 
ADVENTURING SKILLS:
Athletics

COMBAT SKILLS:
Might Defense
Might Attack
Speed Attack
Heavy Weapons

CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:


In [8]:
subotai = Character('Subotai', 14, 22, 12, 0, 3, 1, ['Stealth', 'Initiative', 'Acrobatics', 'Speed Defense', 'Light Weapons', 'Numenera', 'Bluff'])
subotai

Name: Subotai
Might: 14/14
Speed: 22/22
Intellect: 12/12
Skills: 
ADVENTURING SKILLS:
Stealth
Acrobatics

COMBAT SKILLS:
Initiative
Speed Defense
Light Weapons

CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:
Bluff

In [9]:
skills.skills['Speed Attack']

Attack: Stat.Speed

In [10]:
subotai.Challenge(10, ['Speed Attack', 'Light Weapons', 'Stealth', 'Perception'], Stat.Speed, 2, 0)

Target number: 18
Subotai rolled 5.


False

In [11]:
subotai

Name: Subotai
Might: 14/14
Speed: 19/22
Intellect: 12/12
Skills: 
ADVENTURING SKILLS:
Stealth
Acrobatics

COMBAT SKILLS:
Initiative
Speed Defense
Light Weapons

CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:
Bluff

In [12]:
Character('foo', 16, 10, 8)

Name: foo
Might: 16/16
Speed: 10/10
Intellect: 8/8
Skills: 
ADVENTURING SKILLS:


COMBAT SKILLS:


CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:


In [13]:
class Monster:
    def __init__(self, name='', level=0, hp=0, armor=0, attacks={'Might': 0, 'Speed': 0, 'Intellect': 0}, defenses={'Might': 0, 'Speed': 0, 'Intellect': 0}):
        self.name = name
        self.level = level
        self.hp = hp
        self.armor = armor
        self.attacks = attacks
    def Attack(self, target, stat):
        attackLevel = self.level+self.attacks[stat]
        target.Defend(attackLevel, stat)

In [16]:
class Creature:
    def __init__(self,
                 name = '',
                 level=0,
                 hp=0,
                 armor=0,
                 inputSkills=[]):
        self.name = name
        self.level = level
        self.hp = hp
        self.armor = armor
        self.skills = []
        for skill in inputSkills:
            if(skills.isSkill(skill[0])):
                self.skills.append(skill)
    def Attack(self, target, stat=Stat.Speed, defenseEffort=0):
        print('{} attacks {}.'.format(self.name, target.name))
        attackLevel = self.level
        for skill in self.skills:
            if(skill[0].name == 'Attack' and skill[0].stat == stat):
                attackLevel = skill[1]
        attackResult = target.Challenge(attackLevel, 
                                        skills=['{} Defense'.format(stat.name)], 
                                        stat=stat,
                                        effort=defenseEffort)
        if(not attackResult):
            target.intellect.reduce(target.speed.reduce(target.might.reduce(self.level)))
        #print(target)
    def isDead(self):
        return self.hp <= 0

In [18]:
monster = Creature(level=5, name="Monster", hp=15)
subotai = Character('Subotai', 14, 22, 12, 0, 3, 1, ['Stealth', 'Initiative', 'Acrobatics', 'Speed Defense', 'Light Weapons', 'Numenera', 'Bluff'])
turn = 0
while(not subotai.isDead() and not monster.isDead()):
    # Player attacks first
    attackEffort = ''
    while(attackEffort not in ['0', '1']):
        attackEffort = input('Attack Effort: [0, 1]? ')
    attackEffort = int(attackEffort)
    subotai.Attack(monster, effort=attackEffort)
    # Player defends from monster
    defenseEffort = ''
    while(defenseEffort not in ['0', '1']):
        defenseEffort = input('Defense Effort: [0, 1]? ')
    defenseEffort = int(defenseEffort)
    monster.Attack(subotai, defenseEffort=defenseEffort)
    print('Monster hp: {}'.format(monster.hp))
print('Battle ended on turn {}.\nWinner: {winner}'.format(turn, winner="Subotai" if monster.isDead() else "Monster"))

Attack Effort: [0, 1]?1
Subotai attacks Monster.
Target number: 9
Subotai rolled 18.
Defense Effort: [0, 1]?0
Monster attacks Subotai.
Target number: 12
Subotai rolled 6.
Monster hp: 13
Attack Effort: [0, 1]?0
Subotai attacks Monster.
Target number: 12
Subotai rolled 7.
Defense Effort: [0, 1]?0
Monster attacks Subotai.
Target number: 12
Subotai rolled 19.
Monster hp: 13
Attack Effort: [0, 1]?1
Subotai attacks Monster.
Target number: 9
Subotai rolled 19.
Defense Effort: [0, 1]?0
Monster attacks Subotai.
Target number: 12
Subotai rolled 9.
Monster hp: 11
Attack Effort: [0, 1]?1
Subotai attacks Monster.
Not enough to spend 1 effort.
Current: 0
Required: 3
Target number: 12
Subotai rolled 7.
Defense Effort: [0, 1]?0
Monster attacks Subotai.
Target number: 12
Subotai rolled 20.
Monster hp: 11
Attack Effort: [0, 1]?0
Subotai attacks Monster.
Target number: 12
Subotai rolled 10.
Defense Effort: [0, 1]?0
Monster attacks Subotai.
Target number: 12
Subotai rolled 13.
Monster hp: 11
Attack Effort

In [41]:
fights = []
for fight in range(1000):
    monster = Creature(level=5, name="Monster", hp=15)
    subotai = Character('Subotai', 14, 22, 12, 0, 3, 1, ['Stealth', 'Initiative', 'Acrobatics', 'Speed Defense', 'Light Weapons', 'Numenera', 'Bluff'])
    turn = 0
    while(not subotai.isDead() and not monster.isDead()):
        turn += 1
        subotai.Attack(monster)
        monster.Attack(subotai, Stat.Speed)
        #print('Monster hp: {}'.format(monster.hp))
    #print('Battle ended on turn {}.\nWinner: {winner}'.format(turn, winner="Subotai" if monster.isDead() else "Monster"))
    fights.append(('{}'.format("Subotai" if monster.isDead() else "Monster"), turn))

Subotai attacks Monster.
Target number: 12
Subotai rolled 16.
Monster attacks Subotai.
Target number: 12
Subotai rolled 7.
Subotai attacks Monster.
Target number: 12
Subotai rolled 16.
Monster attacks Subotai.
Target number: 12
Subotai rolled 19.
Subotai attacks Monster.
Target number: 12
Subotai rolled 1.
Monster attacks Subotai.
Target number: 12
Subotai rolled 7.
Subotai attacks Monster.
Target number: 12
Subotai rolled 20.
Monster attacks Subotai.
Target number: 12
Subotai rolled 12.
Subotai attacks Monster.
Target number: 12
Subotai rolled 13.
Monster attacks Subotai.
Target number: 12
Subotai rolled 3.
Subotai attacks Monster.
Target number: 12
Subotai rolled 12.
Monster attacks Subotai.
Target number: 12
Subotai rolled 5.
Subotai attacks Monster.
Target number: 12
Subotai rolled 1.
Monster attacks Subotai.
Target number: 12
Subotai rolled 16.
Subotai attacks Monster.
Target number: 12
Subotai rolled 5.
Monster attacks Subotai.
Target number: 12
Subotai rolled 10.
Subotai attacks

In [43]:
fights = []

In [42]:
subotaiWins = 0
monsterWins = 0
turns = 0
for fight in fights:
    if(fight[0] == 'Monster'):
        monsterWins += 1
    elif(fight[0] == 'Subotai'):
        subotaiWins += 1
    turns += fight[1]
averageTurns = turns/len(fights)
print('Subotai wins: {}\nMonster wins: {}\nAverage turns per fight: {}'.format(subotaiWins, monsterWins, averageTurns))

Subotai wins: 583
Monster wins: 417
Average turns per fight: 15.589


In [203]:
monster.Attack(subotai, Stat.Speed)

Target number: 12
Subotai rolled 4.
Name: Subotai
Might: 0/14
Speed: 0/22
Intellect: 0/12
Skills: 
ADVENTURING SKILLS:
Stealth
Acrobatics

COMBAT SKILLS:
Initiative
Speed Defense
Light Weapons

CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:
Bluff



In [16]:
foo.stat.name

'Might'

In [1]:
import sys

sys.path.append('..')

import Characters

sheetID = '1Zni4ySL3zN_SR7DNs-3XQRrya4KZ3RY6C2VtRz6JC_k'

Characters.LoadCharacter(sheetID)

Name: Subotai
Might: 14/14
Speed: 22/22
Intellect: 12/12
Skills: 
ADVENTURING SKILLS:
Stealth
Acrobatics

COMBAT SKILLS:
Initiative
Speed Defense
Light Weapons

CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:
Bluff